<a href="https://colab.research.google.com/github/rutuja-patil24/CMPE-258-Deep_Learning/blob/main/Assignment_05/01_Simple_Text_classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ Install required libraries
!pip install --quiet tensorflow datasets numpy

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from datasets import load_dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-c

In [2]:
# ✅ Load IMDb Dataset
dataset = load_dataset("imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
# ✅ Convert dataset to Pandas DataFrame
df_train = pd.DataFrame(dataset["train"])
df_test = pd.DataFrame(dataset["test"])

In [4]:
# ✅ Preprocess Data
train_texts = df_train["text"].values
train_labels = df_train["label"].values
test_texts = df_test["text"].values
test_labels = df_test["label"].values

In [5]:
# ✅ Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

In [6]:
# ✅ Padding Sequences
MAX_LENGTH = 200
train_padded = pad_sequences(train_sequences, maxlen=MAX_LENGTH, padding="post", truncating="post")
test_padded = pad_sequences(test_sequences, maxlen=MAX_LENGTH, padding="post", truncating="post")

In [7]:
# ✅ Convert labels to NumPy
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [8]:
# ✅ Define Simple LSTM Model
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=MAX_LENGTH),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(16, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")  # Binary Classification
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [9]:
# ✅ Compile Model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
# ✅ Train Model
model.fit(train_padded, train_labels, validation_data=(test_padded, test_labels), epochs=3, batch_size=64)

Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 18s 25ms/step - accuracy: 0.5065 - loss: 0.6930 - val_accuracy: 0.5422 - val_loss: 0.6897
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.5232 - loss: 0.6919 - val_accuracy: 0.5113 - val_loss: 0.6903
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.5129 - loss: 0.6909 - val_accuracy: 0.5171 - val_loss: 0.6912


In [11]:
# ✅ Function to Predict Sentiment
def predict_sentiment(texts):
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=MAX_LENGTH, padding="post", truncating="post")
    predictions = model.predict(padded_sequences)

    sentiment_labels = ["Negative", "Positive"]
    for text, pred in zip(texts, predictions):
        print(f"🎬 Review: {text[:100]}... → **Sentiment: {sentiment_labels[int(pred > 0.5)]}**")

In [12]:
# ✅ Example Predictions
sample_reviews = [
    "This movie was amazing! The story and visuals were fantastic!",
    "Absolutely terrible. One of the worst movies I've ever seen.",
    "It was decent, but nothing too special."
]

predict_sentiment(sample_reviews)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
🎬 Review: This movie was amazing! The story and visuals were fantastic!... → **Sentiment: Negative**
🎬 Review: Absolutely terrible. One of the worst movies I've ever seen.... → **Sentiment: Negative**
🎬 Review: It was decent, but nothing too special.... → **Sentiment: Negative**


<ipython-input-11-9f494ffff7d8>:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"🎬 Review: {text[:100]}... → **Sentiment: {sentiment_labels[int(pred > 0.5)]}**")
